In [22]:
from pathlib import Path
import os

print(f"Current working directory: {os.getcwd()}")
print(f"Notebook location: {Path.cwd()}")

# Test different parent levels
BASE_DIR_2 = Path.cwd().parent.parent
BASE_DIR_3 = Path.cwd().parent.parent.parent
BASE_DIR_4 = Path.cwd().parent.parent.parent.parent

print(f"\nparents[2]: {BASE_DIR_2}")
print(f"parents[3]: {BASE_DIR_3}")
print(f"parents[4]: {BASE_DIR_4}")

# Check which one has 'data/processed/prices_long.csv'
for i, base in enumerate([BASE_DIR_2, BASE_DIR_3, BASE_DIR_4], start=2):
    test_file = base / "data" / "processed" / "prices_long.csv"
    print(f"\nparents[{i}]/data/processed/prices_long.csv exists: {test_file.exists()}")
    print(f"  → {test_file}")

Current working directory: /files/capstone_project/game-market-event-analyzer/src/eventstudy/features
Notebook location: /files/capstone_project/game-market-event-analyzer/src/eventstudy/features

parents[2]: /files/capstone_project/game-market-event-analyzer/src
parents[3]: /files/capstone_project/game-market-event-analyzer
parents[4]: /files/capstone_project

parents[2]/data/processed/prices_long.csv exists: False
  → /files/capstone_project/game-market-event-analyzer/src/data/processed/prices_long.csv

parents[3]/data/processed/prices_long.csv exists: True
  → /files/capstone_project/game-market-event-analyzer/data/processed/prices_long.csv

parents[4]/data/processed/prices_long.csv exists: False
  → /files/capstone_project/data/processed/prices_long.csv


In [23]:
from pathlib import Path
import pandas as pd

# Use Path.cwd() instead of __file__ in notebooks
BASE_DIR = Path.cwd().parent.parent.parent

DATA_RAW = BASE_DIR / "data" / "raw"
DATA_PROCESSED = BASE_DIR / "data" / "processed"

print(f"Project root: {BASE_DIR}")
print(f"Data processed: {DATA_PROCESSED}")

prices = pd.read_csv(DATA_PROCESSED / "prices_long.csv")
prices["date"] = pd.to_datetime(prices["date"])
prices["ticker"] = prices["ticker"].astype(str).str.upper()

print(f"\n✅ Loaded {len(prices)} records")
prices.head()

Project root: /files/capstone_project/game-market-event-analyzer
Data processed: /files/capstone_project/game-market-event-analyzer/data/processed

✅ Loaded 16620 records


,date,ticker,adj_close
0,2015-01-02,ATVI,45.614056
1,2015-01-05,ATVI,45.322094
2,2015-01-06,ATVI,44.728443
3,2015-01-07,ATVI,45.555664
4,2015-01-08,ATVI,46.840290


In [24]:
prices["ticker"] = (
    prices["ticker"]
    .str.replace("^GSPC", "SP500", regex=False)
    .str.replace("UBI.PA", "UBI", regex=False)
    .str.upper()
)

prices["ticker"].unique()

array(['ATVI', 'EA', 'NTDOY', 'TTWO', 'UBI', 'SP500'], dtype=object)

In [25]:
prices = prices.sort_values(["ticker", "date"])
prices["return"] = prices.groupby("ticker")["adj_close"].pct_change()

prices.head(10)

/tmp/ipykernel_10055/960365926.py:2: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  prices["return"] = prices.groupby("ticker")["adj_close"].pct_change()


,date,ticker,adj_close,return
0,2015-01-02,ATVI,45.614056,NaN
1,2015-01-05,ATVI,45.322094,-0.006401
2,2015-01-06,ATVI,44.728443,-0.013098
3,2015-01-07,ATVI,45.555664,0.018494
4,2015-01-08,ATVI,46.840290,0.028199
5,2015-01-09,ATVI,46.236904,-0.012882
6,2015-01-12,ATVI,45.798969,-0.009472
7,2015-01-13,ATVI,45.176105,-0.013600
8,2015-01-14,ATVI,45.643257,0.010341
9,2015-01-15,ATVI,45.390217,-0.005544


In [26]:
# Market ticker (must match what you used above)
market_ticker = "SP500"

market = prices[prices["ticker"] == market_ticker][["date", "return"]].rename(
    columns={"return": "market_return"}
)

prices = prices.merge(market, on="date", how="left")

prices.head(10)

,date,ticker,adj_close,return,market_return
0,2015-01-02,ATVI,45.614056,NaN,NaN
1,2015-01-05,ATVI,45.322094,-0.006401,-0.018278
2,2015-01-06,ATVI,44.728443,-0.013098,-0.008893
3,2015-01-07,ATVI,45.555664,0.018494,0.011630
4,2015-01-08,ATVI,46.840290,0.028199,0.017888
5,2015-01-09,ATVI,46.236904,-0.012882,-0.008404
6,2015-01-12,ATVI,45.798969,-0.009472,-0.008094
7,2015-01-13,ATVI,45.176105,-0.013600,-0.002579
8,2015-01-14,ATVI,45.643257,0.010341,-0.005813
9,2015-01-15,ATVI,45.390217,-0.005544,-0.009248


In [29]:
prices.to_csv(DATA_PROCESSED / "prices_with_returns.csv", index=False)
print(f"✅ Saved to: {DATA_PROCESSED / 'prices_with_returns.csv'}")
print(len(prices), prices.columns)

✅ Saved to: /files/capstone_project/game-market-event-analyzer/data/processed/prices_with_returns.csv
16620 Index(['date', 'ticker', 'adj_close', 'return', 'market_return'], dtype='object')
